In [1]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
PATH = '/content/drive/My Drive/dacon_235658/'

In [3]:
!pip install autokeras
!pip install keras-tuner

     |████████████████████████████████| 174kB 16.7MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=e14cb8403aac08ef5a32570adb909a8523f0a72205b90324f517a347d9dc066f
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=b7746c960cfce49655253e0270b7635ad2ea8405ce18329239ea68b4d59cc68d
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline 
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
train = pd.read_csv(PATH + "input/news_train.csv")
test = pd.read_csv(PATH + "input/news_test.csv")
submission = pd.read_csv(PATH + "input/sample_submission.csv")

In [6]:
train.head()

,n_id,date,title,content,ord,info
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0


In [7]:
train_x = train['content'].values
train_y = train['info'].values

In [8]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
output_node = ak.ConvBlock()(output_node)
output_node = ak.RNNBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=10
)

clf.fit(train_x, train_y, epochs=100)
model = clf.export_model()

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
expand_last_dim (ExpandLastD (None, 1)                 0         
_________________________________________________________________
text_vectorization (TextVect (None, 64)                0         
_________________________________________________________________
embedding (Embedding)        (None, 64, 128)           2560128   
_________________________________________________________________
dropout (Dropout)            (None, 64, 128)           0         
_________________________________________________________________
separable_conv1d (SeparableC (None, 58, 32)            5024      
_________________________________________________________________
separable_conv1d_1 (Separabl (None, 52, 32)            1280  

In [10]:
pred_test = model.predict(test['content'].values)
pred_test

array([[1.5120349e-05],
       [6.8573113e-06],
       [5.6297587e-05],
       ...,
       [9.9985147e-01],
       [9.9985147e-01],
       [9.9985147e-01]], dtype=float32)

In [ ]:
for i in range(len(train)):
    if train_x[i][0] == '[':
        pred_test[i] = 1
    if train_x[i][-1] == ']' or train_x[i][-1] == ')' or train_x[i][0] == '\'' or train_x[i][-1] == '\'' or 48 <= ord(train_x[i][-1]) <= 57:
        pred_test[i] = 0

In [ ]:
submission.loc[:, 'info'] = np.where(pred_test > 0.5, 1, 0).reshape(-1)

In [13]:
submission.loc[:, ['id', 'info']].to_csv(PATH + "output/v1.0.1.csv", index=False)
submission

,id,info
0,NEWS00237_1,1
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,1
142561,NEWS09482_73,1
142562,NEWS09482_74,1
142563,NEWS09482_75,1
